In [ ]:
%load_ext autoreload
%autoreload 2


In [ ]:
from pyprojroot import here

import llamabot as lmb

image_path = here() / "data" / "croc-capybara-alligator.jpg"


messages = lmb.user("what is in this image?", image_path)
bot = lmb.SimpleBot(
    model_name="gpt-4o",
    system_prompt=lmb.system("You are a helpful assistant."),
)

bot(lmb.user("What's in this image?", image_path))

In [ ]:
from pydantic import BaseModel, Field
from datetime import datetime
from decimal import Decimal
from typing import Optional, List


class ReceiptItem(BaseModel):
    """A single item entry in a receipt."""

    name: str = Field(..., description="Name of the item purchased")
    quantity: float = Field(..., description="Quantity of items purchased")
    unit_price: Decimal = Field(..., description="Price per unit of the item")
    total_price: Decimal = Field(..., description="Total price for this item")


class Receipt(BaseModel):
    """A model representing a receipt with its items and metadata."""

    merchant_name: str = Field(..., description="Name of the merchant/store")
    date: datetime = Field(..., description="Date and time of purchase")
    items: List[ReceiptItem] = Field(
        default_factory=list, description="List of items purchased"
    )
    subtotal: Decimal = Field(..., description="Subtotal before tax")
    tax: Decimal = Field(..., description="Tax amount")
    total: Decimal = Field(..., description="Total amount including tax")
    payment_method: Optional[str] = Field(None, description="Method of payment used")
    receipt_number: Optional[str] = Field(
        None, description="Receipt identification number"
    )


sb = lmb.StructuredBot(
    system_prompt=lmb.system("You extract structured information from images."),
    model_name="gpt-4o",
    pydantic_model=Receipt,
)

response = sb(
    lmb.user(
        "Give me only the most expensive items in this image:",
        "https://i.pinimg.com/736x/2d/c9/54/2dc95482e91453fb8469d3b6208a2e3a.jpg",
    )
)